# install & import & load data

In [ ]:
#check GPU 
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Aug 31 17:19:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datasets
#มาจาก huggingface

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import (
    AutoTokenizer, 
    AutoModel,
    AutoModelForSequenceClassification, 
    AutoConfig,

    Trainer, 
    TrainingArguments,
)

from datasets import (
    Dataset,
    DatasetDict,
    load_dataset,
    Features, Sequence, ClassLabel, Value
)

import pandas as pd

# clean & prepare data

In [ ]:
!gdown 1mTfs7CvOba-kbnW1g3RPplEPNRrJR2Vy
!gdown 1hiY6w0k0nzN0gLuhDJbrVWEPhYNeE-kq
!gdown 17tDZEoW7NUikFid-pf5HLap3c_59BOrH

Downloading...
From: https://drive.google.com/uc?id=1mTfs7CvOba-kbnW1g3RPplEPNRrJR2Vy
To: /content/LST_dev.txt
100% 2.96M/2.96M [00:00<00:00, 151MB/s]
Downloading...
From: https://drive.google.com/uc?id=1hiY6w0k0nzN0gLuhDJbrVWEPhYNeE-kq
To: /content/LST_train.txt
100% 32.4M/32.4M [00:00<00:00, 243MB/s]
Downloading...
From: https://drive.google.com/uc?id=17tDZEoW7NUikFid-pf5HLap3c_59BOrH
To: /content/LST_test.txt
100% 2.45M/2.45M [00:00<00:00, 240MB/s]


In [ ]:
import numpy as np
train = pd.read_csv('LST_train.txt', encoding='utf-8',sep='\t')
dev = pd.read_csv('LST_dev.txt', encoding='utf-8', sep='\t')
test = pd.read_csv('LST_test.txt', encoding='utf-8', sep='\t')
def get_label(df):
    star_to_label = {'politics': 0, 'C&A': 1, 'general': 2, 'economics': 3, 'culture': 4, 'STE': 5,
       'international': 6, 'environment': 7, 'weather': 8, 'health': 9 , 'disaster': 10,
       'entertainment': 11 , 'development': 12 , 'sports': 13 , 'royal': 14}
    # apply functions & convert to np.array
    label = np.array(df['lable'].replace(star_to_label).tolist())
    df['label'] = label
    df['text'] = df['tokens'].apply(lambda x: x.replace('|', ''))
    df['texts'] = df['text'].apply(lambda x: x.replace('_', ' '))
    df = df.drop(columns=['tokens', 'num', 'Unnamed: 0', 'text'])
    df = df.rename({'lable' : 'category'}, axis = 'columns')
    return df
    
train = get_label(train)
dev = get_label(dev)
test = get_label(test)    


In [ ]:
train[:10]

,category,label,texts
0,politics,0,สุรยุทธ์ยันปฏิเสธลงนาม MOU กับอียูไม่กระทบสัมพ...
1,C&A,1,บุกยึดไม้เถื่อนอดีตส.ส.บุรีรัมย์เตรียมสร้างคฤห...
2,C&A,1,"ผกก.แจง""สุไฮมี""ที่ถูกวิสามัญ เป็นหนึ่งในมือวาง..."
3,general,2,เฒ่าวัย 72 ร้องถูกรัฐยึดที่ทำกินที่มีสค.1-เป็น...
4,C&A,1,เมียสาวใหญ่สุดแค้นผัวนอกใจมอมเหล้า-เฉือนเจ้าโล...
5,C&A,1,ระแวงแฟนหนุ่มหวนคืนดีอดีตแฟนสบโอกาสตัดจู๋หวิดข...
6,C&A,1,สาวเครียดเจ้าบ่าวปันใจซดยาพิษดับเขียนจม.สั่งแต...
7,C&A,1,มือมืดโทรฯขู่วางระเบิด 5 โรงเรียนย่าน1 สิงหาคม...
8,C&A,1,นราฯวุ่นโจรใต้บึ้ม 6 จุดเจ็บ 2 ยะลายิงชุดรปภ.ร...
9,C&A,1,ยธ.เสนอตั้งกก.อิสระรื้อคดีทักษิณฆ่าตัดตอน 2500...


In [ ]:
train.groupby(['category'])['category'].count()

category
C&A               557
STE               217
culture            34
development        42
disaster           43
economics         481
entertainment     421
environment        36
general            64
health             77
international     228
politics         1178
royal              45
sports            341
weather            30
Name: category, dtype: int64

In [ ]:
train['label'].unique()
labels = ['politics', 'C&A', 'general', 'economics', 'culture', 'STE',
       'international', 'environment', 'weather', 'health', 'disaster',
       'entertainment', 'development', 'sports', 'royal']
label_list = labels

# download model

In [ ]:
#download model 
model_name = 'airesearch/wangchanberta-base-att-spm-uncased'
config = AutoConfig.from_pretrained(model_name, num_labels=len(labels))
tokenizer = AutoTokenizer.from_pretrained(model_name) #tokenizer ของ model นี้ 
model = AutoModelForSequenceClassification.from_pretrained(model_name, 
                                                           config=config)

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForSequenceClassification: ['lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wa

In [ ]:
features= Features({
                    #"tweet text": Sequence(Value(dtype='string')),
                    'news text':  Value(dtype='string'),

                    'label': ClassLabel(names=label_list)
                })

d = DatasetDict({'train': Dataset.from_pandas(train), 
                 'dev': Dataset.from_pandas(dev),
                 'test': Dataset.from_pandas(test)})

In [ ]:
d['train'][1]

{'category': 'C&A',
 'label': 1,
 'texts': 'บุกยึดไม้เถื่อนอดีตส.ส.บุรีรัมย์เตรียมสร้างคฤหาสน์ทรงไทย1 กันยายน 2550 12:00 น.ตำรวจภูธรจ.บุรีรัมย์บุกตรวจยึดไม้แปรรูปหวงห้ามกว่า 80 แผ่นเตรียมนำไปก่อสร้างคฤหาสน์ทรงไทยของอดีตส.ส.ดังพร้อมนำตัวภายในบ้านไปสอบสวนเพื่อให้เจ้าของนำเอกสารหลักฐานที่มาของไม้มาแสดงสายวันนี้(1 ก.ย.) พ.ต.อ.สังวรณ์ ภู่ไพจิตกุล รองผบก.ภ.จว.บุรีรัมย์พร้อมพ.ต.ท.สุธีร์ ตรุโนภาส รอง ผกก.สภ.อ.เมืองสว.สส.และร.ต.ท.ภานุวัฒน์ มากมูล ร้อยเวรฯสภ.อ.เมืองทำการตรวจสอบไม้แปรรูปหวงห้ามเตรียมนำไปก่อสร้างคฤหาสน์ทรงไทยหรูภายในฟาร์มเลี้ยงวัวเอ็ม จี ประสิทธิ์ฟาร์ม บ.หนองไผ่น้อย ต.ชุมเห็ด อ.เมืองได้รับแจ้งจากชาวบ้านว่ามีการลักลอบแปรรูปไม้หวงห้ามในบริเวณดังกล่าวจากการตรวจสอบในโกดังที่อยู่ด้านหลังฟาร์มพบไม้ประดู่ซึ่งเป็นไม้หวงห้ามที่แปรรูปแล้วจำนวน 81 แผ่น เจ้าหน้าที่จึงได้ทำการตรวจยึดไว้ตรวจสอบพร้อมทั้งได้นำตัวสุพจน์ โพธิ์เงินและนายศิรินดง แก่นใสภายในฟาร์มไปสอบสวนที่สภ.อ.เมือง 2 ให้การว่าไม่ทราบของไม้ส่วนบ้านทรงไทยที่ปลูกสร้างในบริเวณฟาร์มเป็นบ้านนายประสิทธิ์ ตั้งศรีเกียรติกุล อดีตส.ส.ดังเมืองบ

In [ ]:
d

DatasetDict({
    train: Dataset({
        features: ['category', 'label', 'texts'],
        num_rows: 3794
    })
    dev: Dataset({
        features: ['category', 'label', 'texts'],
        num_rows: 474
    })
    test: Dataset({
        features: ['category', 'label', 'texts'],
        num_rows: 483
    })
})

In [ ]:
#tokenizer 
def tokenize(examples):
    tokenized_inputs = tokenizer(examples["texts"], 
                                 is_split_into_words=False,
                                 truncation=True,
                                 max_length=40)
    return tokenized_inputs
tokenized_datasets = d.map(tokenize, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][10]['input_ids'])

['<s>',
 '▁',
 'ธีร',
 'ภัทร์',
 'แนะ',
 '"',
 'สมัคร',
 '"',
 'เคลียร์',
 'คดี',
 'ก่อน',
 'เป็น',
 'หน',
 '.',
 'พรรคพลัง',
 'ปชช',
 '.',
 '1',
 '▁',
 'สิงหาคม',
 '▁',
 '2550',
 '▁',
 '11:',
 '53',
 '▁',
 'น',
 '.',
 'ธีร',
 'ภัทร์',
 'แขวะ',
 'สมัคร',
 'เคลียร์',
 'คดี',
 'รถดับเพลิง',
 'ฉาว',
 'ก่อน',
 'ค่อย',
 'คิดเป็น',
 '</s>']

In [ ]:
model_name.split('/')[-1]

'wangchanberta-base-att-spm-uncased'

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['category', 'label', 'texts', 'input_ids', 'attention_mask'],
        num_rows: 3794
    })
    dev: Dataset({
        features: ['category', 'label', 'texts', 'input_ids', 'attention_mask'],
        num_rows: 474
    })
    test: Dataset({
        features: ['category', 'label', 'texts', 'input_ids', 'attention_mask'],
        num_rows: 483
    })
})

In [ ]:
#drop tweet text ทิ้ง
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer,
                                        padding=True, pad_to_multiple_of=8)
tokenized_datasets = tokenized_datasets.remove_columns(['texts'])
tokenized_datasets = tokenized_datasets.remove_columns(['category'])
#tokenized_datasets = tokenized_datasets.remove_columns(['__index_level_0__'])
#tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
#tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['label', 'input_ids', 'attention_mask']

In [ ]:
new = pd.DataFrame()
new['input_len'] = [len(x) for x in tokenized_datasets['train']['input_ids']]

c = 0
for num in new['input_len']:
    if num > 40:
      c += 1
print(c)

new.describe()

0


,input_len
count,3794.000000
mean,39.998419
std,0.097410
min,34.000000
25%,40.000000
50%,40.000000
75%,40.000000
max,40.000000


In [ ]:
tokenized_datasets['dev'][0]

{'label': 10,
 'input_ids': [5,
  10,
  8294,
  17154,
  9367,
  111,
  598,
  8805,
  2813,
  67,
  288,
  10,
  22634,
  4277,
  35,
  1350,
  291,
  11699,
  111,
  1372,
  8805,
  505,
  2813,
  67,
  288,
  1272,
  150,
  6183,
  1305,
  10,
  1568,
  1124,
  600,
  10888,
  12,
  963,
  67,
  2918,
  2648,
  6],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [ ]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items()}
[len(x) for x in samples["input_ids"]]

[40, 40, 40, 40, 40, 40, 40, 40]

In [ ]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 40]),
 'attention_mask': torch.Size([8, 40]),
 'labels': torch.Size([8])}

In [ ]:
from datasets import list_metrics, load_metric
list_metrics()

['accuracy',
 'bertscore',
 'bleu',
 'bleurt',
 'brier_score',
 'cer',
 'chrf',
 'code_eval',
 'comet',
 'competition_math',
 'coval',
 'cuad',
 'exact_match',
 'f1',
 'frugalscore',
 'glue',
 'google_bleu',
 'indic_glue',
 'mae',
 'mahalanobis',
 'matthews_correlation',
 'mauve',
 'mean_iou',
 'meteor',
 'mse',
 'pearsonr',
 'perplexity',
 'poseval',
 'precision',
 'recall',
 'rl_reliability',
 'roc_auc',
 'rouge',
 'sacrebleu',
 'sari',
 'seqeval',
 'spearmanr',
 'squad',
 'squad_v2',
 'super_glue',
 'ter',
 'trec_eval',
 'wer',
 'wiki_split',
 'xnli',
 'xtreme_s',
 'Vertaix/vendiscore',
 'Vlasta/pr_auc',
 'abdusahmbzuai/aradiawer',
 'angelina-wang/directional_bias_amplification',
 'cakiki/ndcg',
 'codeparrot/apps_metric',
 'cpllab/syntaxgym',
 'daiyizheng/valid',
 'erntkn/dice_coefficient',
 'gorkaartola/metric_for_tp_fp_samples',
 'hack/test_metric',
 'idsedykh/codebleu',
 'idsedykh/codebleu2',
 'idsedykh/megaglue',
 'idsedykh/metric',
 'jordyvl/ece',
 'kadirnar/bary_score',
 'kagg

In [ ]:
load_metric('accuracy')

Metric(name: "accuracy", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = datasets.load_metric("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
        {'accuracy': 0.5}

   

# train model

In [ ]:
args = TrainingArguments(
    output_dir='prachatai-headline'
)

#ปรับตรงนี้ !!!
args2 = TrainingArguments(
    output_dir='prachatai-headline-2', #เอาไปแขวนไว้ที่ไหน
    learning_rate=1e-4,
    per_device_train_batch_size=128, #batch size 
    per_device_eval_batch_size=128,
    num_train_epochs=12, #epoch
    load_best_model_at_end=True, #เอาอันที่ดีที่สุดจาก accuracy 
    metric_for_best_model='accuracy',
    evaluation_strategy='steps', #evaluate ทุก 100 steps OR epochs ก็ได้ แต่อย่ารันบ่อยเพราะเสียเวลา 
    eval_steps=50, 
    #save_strategy='epoch'    
)
#metric.compute(predictions=preds, references=predictions.label_ids)
import numpy as np

def compute_metrics(eval_preds):
  metric = load_metric("accuracy")
  logits, labels = eval_preds #logists = probability แต่ละ label
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

#train model 
trainer = Trainer(
    model,
    args2,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3794
  Num Epochs = 12
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 360


Step,Training Loss,Validation Loss,Accuracy
50,No log,0.791152,0.776371
100,No log,0.538836,0.827004
150,No log,0.647518,0.839662
200,No log,0.598302,0.852321
250,No log,0.709092,0.858650
300,No log,0.689159,0.869198
350,No log,0.728502,0.862869


***** Running Evaluation *****
  Num examples = 474
  Batch size = 128
***** Running Evaluation *****
  Num examples = 474
  Batch size = 128
***** Running Evaluation *****
  Num examples = 474
  Batch size = 128
***** Running Evaluation *****
  Num examples = 474
  Batch size = 128
***** Running Evaluation *****
  Num examples = 474
  Batch size = 128
***** Running Evaluation *****
  Num examples = 474
  Batch size = 128
***** Running Evaluation *****
  Num examples = 474
  Batch size = 128


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=360, training_loss=0.38454623752170136, metrics={'train_runtime': 314.9447, 'train_samples_per_second': 144.559, 'train_steps_per_second': 1.143, 'total_flos': 935962369453440.0, 'train_loss': 0.38454623752170136, 'epoch': 12.0})

In [ ]:
predictions = trainer.predict(tokenized_datasets['test'])
preds = np.argmax(predictions.predictions, axis=-1)

***** Running Prediction *****
  Num examples = 483
  Batch size = 128


In [ ]:
predictions.label_ids

array([10, 10, 10,  4,  4, 14,  2,  2,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6, 14,  2, 14, 10,  6,  6,  6,  6,  9,  6, 13,
       13, 13,  6,  9,  6,  6,  9, 13, 13, 13, 13, 13, 13, 13,  9,  9, 13,
       13, 13, 13, 13, 13, 13, 13,  6, 13,  9,  7,  8, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11,  8,  9, 11,  7,  6, 13, 13,  3,  3, 11,  3, 11,
        6,  3,  6,  3,  3,  8,  3,  3,  3,  3, 12,  3,  2, 11, 11, 11, 11,
       11, 11, 13, 13, 13, 13, 13, 11,  3, 12,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3, 11, 11, 11,  6, 13, 13, 13, 13, 13, 13, 13, 13,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  3,  3,  3,  3,  3,
       11, 11, 11, 11, 11, 11, 13, 13, 13, 13, 13, 13, 13,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  3,  3,  3, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 13, 13,  1,  1,  1,  1,  1,  1,  1,  3,  3,
        5,  5,  5, 11, 11, 11, 11, 11, 11, 11, 11,  1,  1,  1,  1,  1,  1,
        5,  3,  1,  1,  1

In [ ]:
preds

array([ 6,  8,  7,  4,  3, 14, 12,  1,  6,  6,  6,  6,  0,  6,  6,  6,  6,
        6,  6,  0,  3,  6,  3, 14,  9, 14, 10,  6,  6,  6,  6,  9,  6, 13,
       13, 13,  3,  9,  6,  6,  0, 13, 13, 13, 13, 13, 13, 13,  3,  9, 13,
       13, 13, 13, 13,  0,  1,  0,  0, 13,  3,  7,  8, 11, 11, 11, 11, 11,
       11,  4, 11, 11, 11,  5,  9, 11,  9,  6, 13, 13,  3,  3, 11, 11, 11,
        3,  3,  6,  3,  3,  8,  0,  3,  3,  3, 12,  3, 11, 11, 11, 11, 11,
       11, 11, 13, 13, 13, 13,  6,  0,  3, 12,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  5, 11, 11, 11,  6, 13, 13, 13,  1, 13, 13, 13, 13,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  0,  1,  1,  1,  5,  3,  3,  3,  3,
       11,  9, 11, 11, 11, 11, 13, 13, 13, 13, 13, 13, 13,  1,  1,  1,  1,
       11,  1,  1,  1, 11,  1,  5,  3,  5, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 13, 13,  1,  1,  1,  1,  1,  1,  1,  3,  5,
        5,  5,  0, 11, 11, 11, 13, 11, 11, 11, 11,  1,  4,  1,  1,  1,  1,
        5,  3,  1,  2,  1

In [ ]:
predicted_labels = [labels[x] for x in preds]
gold = [labels[x] for x in predictions.label_ids]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(gold, predicted_labels))

               precision    recall  f1-score   support

          C&A       0.91      0.83      0.87        70
          STE       0.68      0.68      0.68        28
      culture       0.33      0.20      0.25         5
  development       0.33      0.33      0.33         6
     disaster       1.00      0.17      0.29         6
    economics       0.73      0.62      0.67        61
entertainment       0.91      0.91      0.91        53
  environment       0.60      0.60      0.60         5
      general       0.00      0.00      0.00         8
       health       0.44      0.70      0.54        10
international       0.88      0.77      0.82        30
     politics       0.79      0.95      0.86       148
        royal       0.71      0.83      0.77         6
       sports       0.97      0.88      0.93        43
      weather       0.67      0.50      0.57         4

     accuracy                           0.80       483
    macro avg       0.66      0.60      0.61       483
 weighte